In [1]:
from bs4 import BeautifulSoup as bs
from lxml import etree
from pprint import pprint
from pyquery import PyQuery as pq
from selenium import webdriver
from urllib.parse import urlencode
import json
import os
import pandas as pd
import re
import requests
import time

In [2]:
headers = {
    'user-agent': 'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'}

In [3]:
region_map = {
    '台北市':'1',
    '基隆市':'2',
    '新北市':'3',
    '新竹市':'4',
    '新竹縣':'5',
    '桃園市':'6',
    '苗栗縣':'7',
    '台中市':'8',
    '彰化縣':'10',
    '南投縣':'11',
    '嘉義市':'12',
    '嘉義縣':'13',
    '雲林縣':'14',
    '台南市':'15',
    '高雄市':'17',
    '屏東縣':'19',
    '宜蘭縣':'21',
    '台東縣':'22',
    '花蓮縣':'23',
    '澎湖縣':'24',
    '金門縣':'25',
    '連江縣':'26',
}

In [4]:
client = requests.session()

In [5]:
def get_var(client,region,headers):
    index_url='https://rent.591.com.tw/?kind=0&regionid='+region_map[region]
    res = client.get(index_url,headers=headers)
    res.encoding = 'utf-8'
    res.cookies['urlJumpIp']= region_map[region]
#     print(res.cookies.get_dict())
    soup = bs(res.text, "lxml")
#     print(soup.select('head > meta'))

    totalRows = soup.select('#container > section.listBox > div > div.listLeft > div.page-limit > div > a.pageNext')[0]['data-total']
    CSRF_TOKEN = [i['content'] for i in soup.select('head > meta[name]') if i['name']=="csrf-token"][0]
    return (totalRows ,CSRF_TOKEN)

In [6]:
def get_searchUrl(firstRow,region,headers):
    params = {
        'is_new_list': 1,
        'type': 1,
        'kind':0,
        'searchtype':1,
        'region':region_map[region],
        'firstRow':(firstRow*30),
        'totalRows':get_var(client,region,headers)[0]
    }
    base_url = 'https://rent.591.com.tw/home/search/rsList?'
    url = base_url + urlencode(params)
    return url

In [7]:
from pprint import pprint

In [8]:
import math

In [9]:
def get_searchUrlist(client,region,headers):
    vars = get_var(client,region,headers)
    headers['X-CSRF-Token'] = vars[1]
    times = math.floor(int(vars[0])/30) +1 if int(vars[0])%30 !=0 else math.floor(int(vars[0])/30)
    searchUrlist=[]
    for firstRow in range(times):
        searchUrl = get_searchUrl(firstRow,region,headers)
#         print(searchUrl)
        res_search = client.get(searchUrl,headers=headers)
        res_search.encoding = 'utf-8'
        data = json.loads(res_search.text)
#         print(data['data']['data'][0]['post_id'])
        rent_list = [[i['post_id'],i['region_name']+' '+i['fulladdress'],'https://rent.591.com.tw/rent-detail-'+str(i['post_id'])+'.html'] for i in data['data']['data']]
#         print(rent_list[0])
        searchUrlist.extend(rent_list)     
    return searchUrlist
        

In [10]:
vars = get_var(client,'花蓮縣',headers)

In [11]:
headers['X-CSRF-Token'] = vars[1]

In [12]:
url = 'https://rent.591.com.tw/home/search/rsList?is_new_list=1&type=1&kind=0&searchtype=1&region=23&firstRow=0&totalRows=394'

In [13]:
url = 'https://rent.591.com.tw/rent-detail-8935215.html'

In [15]:
res= client.get(url,headers=headers)

In [17]:
res.encoding = 'utf-8'

In [18]:
soup_detail = bs(res_detail.text, "lxml")

In [18]:
data = json.loads(res.text)

In [22]:
data['data']['data']

[{'addInfo': '',
  'addition2': 0,
  'addition3': 0,
  'addition4': 1,
  'addr_number_name': '',
  'address': '國聯四路步行三分鐘..',
  'address_img': '步行三分鐘到火車站，有大廈管理員',
  'address_img_title': '步行三分鐘到火車站，有大廈管理員',
  'alley_name': '',
  'allfloor': 11,
  'area': 7,
  'balcony_area': None,
  'browsenum': 91,
  'browsenum_all': 5985,
  'browsenum_name': '昨日瀏覽',
  'cartplace': '',
  'cases_id': 0,
  'cases_name': '',
  'checkstatus': 0,
  'closed': 0,
  'comment_class': '',
  'comment_ltime': 1583216711,
  'comment_total': 2,
  'comment_unread': 0,
  'condition': 'tv,icebox,cold,washer,hotwater,broadband,lift,landpost,balcony_0,lease,bed,wardrobe,bookTable,chair,all_sex',
  'cover': 'https://hp2.591.com.tw/house/active/2019/07/09/156267832720403103_210x158.crop.jpg',
  'distance': 0,
  'filename': 'https://hp2.591.com.tw/house/active/2019/07/09/156267832720403103_210x158.crop.jpg',
  'floor': 4,
  'floor2': 0,
  'floorInfo': '樓層：4/11',
  'fulladdress': '國聯四路步行三分鐘到火車站，有大廈管理員',
  'groundarea': None,


In [63]:
a = [[i['post_id'],i['region_name']+' '+i['fulladdress'],'https://rent.591.com.tw/rent-detail-'+str(i['post_id'])+'.html'] for i in data['data']['data']][0]

In [64]:
a

[8935215,
 '花蓮縣 延平街148巷正市區。近花商、85度c',
 'https://rent.591.com.tw/rent-detail-8935215.html']

In [50]:
c

dict_keys([8935215])

In [11]:
%time searchUrl = get_searchUrlist(client,'花蓮縣',headers)

https://rent.591.com.tw/home/search/rsList?is_new_list=1&type=1&kind=0&searchtype=1&region=23&firstRow=0&totalRows=394
https://rent.591.com.tw/home/search/rsList?is_new_list=1&type=1&kind=0&searchtype=1&region=23&firstRow=30&totalRows=394
https://rent.591.com.tw/home/search/rsList?is_new_list=1&type=1&kind=0&searchtype=1&region=23&firstRow=60&totalRows=394
https://rent.591.com.tw/home/search/rsList?is_new_list=1&type=1&kind=0&searchtype=1&region=23&firstRow=90&totalRows=394
https://rent.591.com.tw/home/search/rsList?is_new_list=1&type=1&kind=0&searchtype=1&region=23&firstRow=120&totalRows=394
https://rent.591.com.tw/home/search/rsList?is_new_list=1&type=1&kind=0&searchtype=1&region=23&firstRow=150&totalRows=394
https://rent.591.com.tw/home/search/rsList?is_new_list=1&type=1&kind=0&searchtype=1&region=23&firstRow=180&totalRows=394
https://rent.591.com.tw/home/search/rsList?is_new_list=1&type=1&kind=0&searchtype=1&region=23&firstRow=210&totalRows=394
https://rent.591.com.tw/home/search/r

In [17]:
len(searchUrl)

394

In [115]:
get_house(searchUrl[:10],headers)[0]

{'name': '葉小姐',
 'dialPhone': '0925-610-909',
 'hid_tel': '03-8533607',
 'hid_email': 'wilun.masa@gmail.com',
 'price': '42,000 元/月 ',
 '押金': '二個月',
 '車位': '平面式停車位，已含租金內',
 '管理費': '--',
 '最短租期': '兩年',
 '開伙': '不可以',
 '養寵物': '不可以',
 '身份要求': '學生、上班族、家庭',
 '朝向': '坐西北朝東南',
 '可遷入日': '隨時',
 '法定用途': '住家用',
 '建物面積': '67.00坪（不含公設）',
 '產權登記': '已辦',
 '格局': '5房2廳5衛5陽台',
 '坪數': '73坪',
 '樓層': '整棟/4F',
 '型態': '別墅',
 '現況': '整層住家'}

In [113]:
def get_house(searchUrl,headers):
    houses=[]
    for url in searchUrl:
        detail_url = url[1]
        res_detail = requests.get(detail_url,headers=headers)
        soup_detail = bs(res_detail.text, "lxml")
        hid_tel =  soup_detail.select('div.userInfo > div > input#hid_tel')[0]['value'] 
        hid_email = soup_detail.select('div.userInfo > div > input#hid_email')[0]['value']
        infs = [{j[0].text.replace(' ',''):j[1].text.replace(' ','').replace('：','') for j in  zip(i.select('div.one'),i.select('div.two'))} for i in soup_detail.select('ul.clearfix.labelList.labelList-1 > li')]
        attrs = [{''.join(i.text.split(':')[0].split()):''.join(i.text.split(':')[1].split()) } for i in soup_detail.select('ul.attr > li')]
        out_rec = {
            'name':soup_detail.select('div.userInfo > div > span.kfCallName')[0]['data-name'],
            'dialPhone' : soup_detail.select('div.userInfo > div > span.dialPhoneNum')[0]['data-value'],
            'hid_tel' : hid_tel if "hid_tel" in locals() else None,
            'hid_email' : hid_email if "hid_email" in locals() else None,
            'price':soup_detail.select('div.price')[0].text.split('\n')[1],
            
        }
        [out_rec.update(inf) for inf in infs]
        [out_rec.update(attr) for attr in attrs]
        houses.append(out_rec)
    return houses
        

In [95]:
s = {'a':1,'b':2}

In [96]:
p = [{'c':3},{'d':4}]

In [98]:
[s.update(i) for i in p]

[None, None]

In [99]:
s

{'a': 1, 'b': 2, 'c': 3, 'd': 4}

In [16]:
searchUrl[0][1]

'https://rent.591.com.tw/rent-detail-8815370.html'

In [18]:
detail_url = searchUrl[0][1]

In [19]:
res_detail = requests.get(detail_url,headers=headers)

In [20]:
soup_detail = bs(res_detail.text, "lxml")

In [51]:
inf = [{j[0].text.replace(' ',''):j[1].text.replace(' ','').replace('：','') for j in  zip(i.select('div.one'),i.select('div.two'))} for i in soup_detail.select('ul.clearfix.labelList.labelList-1 > li')]
inf

[{'押金': '二個月'},
 {'車位': '平面式停車位，已含租金內'},
 {'管理費': '--'},
 {'最短租期': '兩年'},
 {'開伙': '不可以'},
 {'養寵物': '不可以'},
 {'身份要求': '學生、上班族、家庭'},
 {'朝向': '坐西北朝東南'},
 {'可遷入日': '隨時'},
 {'法定用途': '住家用'},
 {'建物面積': '67.00坪（不含公設）'},
 {'產權登記': '已辦'}]

In [49]:
# price = soup_detail.select('div.price')[0].text.split('\n')[1]
# price

'42,000 元/月 '

In [60]:
attrs = [{''.join(i.text.split(':')[0].split()):''.join(i.text.split(':')[1].split()) } for i in soup_detail.select('ul.attr > li')]
attrs

[{'格局': '5房2廳5衛5陽台'},
 {'坪數': '73坪'},
 {'樓層': '整棟/4F'},
 {'型態': '別墅'},
 {'現況': '整層住家'}]

In [71]:
name = soup_detail.select('div.userInfo > div > span.kfCallName')[0]['data-name']

In [76]:
dialPhone = soup_detail.select('div.userInfo > div > span.dialPhoneNum')[0]['data-value']

In [79]:
hid_tel = soup_detail.select('div.userInfo > div > input#hid_tel')[0]['value']

In [83]:
hid_email = soup_detail.select('div.userInfo > div > input#hid_email')[0]['value']

In [108]:
if "hid_tel" in locals():
    print('a')

a
